## Resnet 18 구현

출처 : [ResNet18_CIFAR10_Train.ipynb](https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb)

## Import required libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os

## Basic block 만들기
- `nn.Conv2d` : 2d convolution layer (input_channel, output_channel, stride, padding, bias)  
- `nn.BatchNorm2d` : Input이 2d 이미지일 때, 사용하는 Batch Normalization 기법  
    Batch Normalization을 사용하는 이유? - layer를 통과해도 이미지 분포에 대한 변화를 처음과 비슷한 분포로 만들기 위해서
    
- `nn.Identity()` : Input과 동일한 값을 ouput으로 보내주기 위해 쓰는 함수
- `nn.Sequential()` : 여러 layer를 쌓을 때 사용하는 함수

In [2]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride = 1):
        super(BasicBlock, self).__init__()
        
        # 3x3 필터를 사용 - 너비와 높이를 줄일 때는 stride 값을 조절
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size = 3, stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)
        
        # 3x3 필터 사용 - 이번에는 너비와 높이가 동일
        self.conv2 = nn.Conv2d(planes,planes,kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Identity() # identity인 경우
        # 차원을 낮춰줘야 하는 경우
        if stride != 1:            
            self.shortcut = nn.Sequential(
                            nn.Conv2d(in_planes, planes, kernel_size = 1, stride = stride, bias = False),
                            nn.BatchNorm2d(planes)
                            )
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # resnet 논문의 핵심!
        out = F.relu(out)
        
        return out

## Basic Block을 여러번 묶어서 Resnet 구현
- `_make_layer` : layer를 block의 수만큼 쌓아 올리는 함수
    - strides = [ stride, 1 , 1, 1, ... ] - (1의 갯수가 num_blocks - 1만큼)
    - layer는 가변인자를 ouput으로 내보내는 것  
        참고 : [파이썬의 Asterisk(*) 이해하기](https://mingrammer.com/understanding-the-asterisk-of-python/)

In [12]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes = 10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        
        # 64개의 3x3 필터를 사용
        self.conv1 = nn.Conv2d(3,64,kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride = 1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride = 2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride = 2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride = 2)
        self.linear = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1) # out.size(0) = out.reshape(0)
        out = self.linear(out)
        
        return out

## DataSet(CIFAR10) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms

In [5]:
train = transforms.Compose([
    transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root= './data', train = True, download = True, transform = train)
test_dataset = torchvision.datasets.CIFAR10(root= './data', train = False, download = True, transform = test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 128, shuffle = True, num_workers = 4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle = False, num_workers = 4)

Files already downloaded and verified
Files already downloaded and verified


## Resnet 18 함수 정의

In [6]:
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

## 환경 설정 및 함수 정의

In [7]:
device = 'cuda'
cudnn.benchmark = True

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)

learning_rate = 0.1
file_name = 'resnet18_cifar10.pth'

# Loss function 정의
criterion = nn.CrossEntropyLoss()
# Optimizer 정의
optimizer = optim.SGD(net.parameters(), lr = learning_rate, momentum = 0.9, weight_decay = 0.0002)

## Train 함수 정의

In [8]:
def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in  enumerate(train_loader):
        inputs= inputs.to(device)
        targets = targets.to(device)
        
        outputs = net(inputs)
        
        optimizer.zero_grad()  # 가중치값 초기화 과정
        loss = criterion(outputs, targets) # Loss 계산
        loss.backward()  # Backpropagation
        optimizer.step() # 가중치 Update
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)

## Test 함수 정의

In [9]:
def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')

## Learning_rate 조절 함수 정의

In [10]:
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

## Training

In [11]:
for epoch in range(0, 20):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.125
Current benign train loss: 2.377624273300171

Current batch: 100
Current benign train accuracy: 0.1640625
Current benign train loss: 2.0662007331848145

Current batch: 200
Current benign train accuracy: 0.34375
Current benign train loss: 1.8738585710525513

Current batch: 300
Current benign train accuracy: 0.296875
Current benign train loss: 1.9341106414794922

Total benign train accuarcy: 25.782
Total benign train loss: 844.2018893957138

[ Test epoch: 0 ]

Test accuarcy: 32.19
Test average loss: 0.018502931606769562
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.25
Current benign train loss: 1.8070673942565918

Current batch: 100
Current benign train accuracy: 0.375
Current benign train loss: 1.6769075393676758

Current batch: 200
Current benign train accuracy: 0.3359375
Current benign train loss: 1.6720048189163208

Current batch: 300
Current benign train accuracy: 0.4375



Current batch: 200
Current benign train accuracy: 0.7890625
Current benign train loss: 0.6133056282997131

Current batch: 300
Current benign train accuracy: 0.8359375
Current benign train loss: 0.48437556624412537

Total benign train accuarcy: 84.706
Total benign train loss: 171.62866127490997

[ Test epoch: 13 ]

Test accuarcy: 77.34
Test average loss: 0.006857270067930222
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.8359375
Current benign train loss: 0.4085725247859955

Current batch: 100
Current benign train accuracy: 0.8515625
Current benign train loss: 0.39944109320640564

Current batch: 200
Current benign train accuracy: 0.828125
Current benign train loss: 0.5140188336372375

Current batch: 300
Current benign train accuracy: 0.8671875
Current benign train loss: 0.41965219378471375

Total benign train accuarcy: 85.614
Total benign train loss: 162.85174638032913

[ Test epoch: 14 ]

Test accuarcy: 75.36
Test average loss: 0.00740808379054069